In [55]:
#테스트 끝나면 py코드로 바꿀 예정

import requests 
url = "https://api.upbit.com/v1/market/all" 
querystring = {"isDetails":"false"} 
response = requests.request("GET", url, params=querystring) 

print(response.json()[0])

{'market': 'KRW-BTC', 'korean_name': '비트코인', 'english_name': 'Bitcoin'}


In [26]:
import time 
import pyupbit
import pandas as pd
import datetime


In [27]:
access = input() # 발급받은 엑세스키(개인)
secret = input() # 발급받은 시크릿키 입력 (개인)
login = pyupbit.Upbit(access, secret)

balance_df = pd.DataFrame(login.get_balances()) # 암호화폐 잔고 조회

print(balance_df) 

Empty DataFrame
Columns: []
Index: []


In [35]:
upbit = pyupbit.Upbit(access, secret)

def get_target_price(ticker, k):
    """변동성 돌파 전략으로 매수 목표가 조회"""
    df = pyupbit.get_ohlcv(ticker, interval="day", count=2)
    target_price = df.iloc[0]['close'] + (df.iloc[0]['high'] - df.iloc[0]['low']) * k
    return target_price

def get_start_time(ticker):
    """시작 시간 조회"""
    df = pyupbit.get_ohlcv(ticker, interval="day", count=1)
    start_time = df.index[0]
    return start_time

def get_balance(ticker):
    """잔고 조회"""
    balances = upbit.get_balances()
    for b in balances:
        if b['currency'] == ticker:
            if b['balance'] is not None:
                return float(b['balance'])
            else:
                return 0
    return 0

    
def get_current_price(ticker):
    """현재가 조회"""
    return pyupbit.get_orderbook(ticker=ticker)["orderbook_units"][0]["ask_price"]


In [67]:
krw = get_balance("KRW") # 계좌 내 한화 잔금

target_price = get_target_price("KRW-DOGE", 0.5)

current_price = get_current_price("KRW-DOGE")

print(krw)
print(current_price)
print(target_price)
# 모든 코인 최소 주문금액 5천원임
upbit.buy_market_order("KRW-DOGE", 1100)

print(krw)

6000.0
184.0
185.5
400
최소주문금액 이상으로 주문해주세요
under_min_total_bid
UpbitError
6000.0


In [2]:
#케이뱅크 계좌 연동 필요

while True:
    try:
        now = datetime.datetime.now()
        start_time = get_start_time("KRW-BTC")
        end_time = start_time + datetime.timedelta(days=1)

        if start_time < now < end_time - datetime.timedelta(seconds=10):
            target_price = get_target_price("KRW-BTC", 0.5)
            current_price = get_current_price("KRW-BTC")
            if target_price < current_price:
                krw = get_balance("KRW")
                if krw > 5000:
                    upbit.buy_market_order("KRW-BTC", krw*0.9995)
                    print("bought KRW-BTC with ", krw*0.9995 )
        else:
            btc = get_balance("BTC")
            if btc > 0.00008:
                upbit.sell_market_order("KRW-BTC", btc*0.9995)
                print("selled KRW-BTC for ", krw*0.9995 )
        time.sleep(1)
    except Exception as e:
        print(e)
        time.sleep(1)

name 'datetime' is not defined


NameError: name 'time' is not defined